### Bias-variance tradeoff
We do this to find the best order of complexity of our model. This is done based on the same method as used in the labs.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2

In [ ]:
from scripts.proj1_helpers import *
DATA_TRAIN_PATH = './data/train.csv' # TODO: download train data and supply path here 
y, tX2, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)

tX2.shape

In [ ]:
from scripts.clean_data import clean_data
tx = clean_data(tX2)

In [ ]:
def least_squares(y, tx):
    """calculate the least squares."""
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)


In [ ]:
def bias_variance_decomposition_visualization(degrees, rmse_tr, rmse_te):
    """visualize the bias variance decomposition."""
    f = plt.figure(figsize = (15,10))
    rmse_tr_mean = np.expand_dims(np.mean(rmse_tr, axis=0), axis=0)
    rmse_te_mean = np.expand_dims(np.mean(rmse_te, axis=0), axis=0)
    """
    plt.plot(
        degrees,
        rmse_tr.T,
        'b',
        linestyle="-",
        color=([0.7, 0.7, 1]),
        label='train',
        linewidth=0.3)
    #plt.plot(
        degrees,
        rmse_te.T,
        'r',
        linestyle="-",
        color=[1, 0.7, 0.7],
        label='test',
        linewidth=0.3)
    plt.plot(
        degrees,
        rmse_tr_mean.T,
        'b',
        linestyle="-",
        label='train',
        linewidth=3) """
    plt.plot(
        degrees,
        rmse_te_mean.T,
        'r',
        linestyle="-",
        label='test',
        linewidth=3)
    plt.xlabel("degree")
    plt.ylabel("error")
    plt.title("Bias-Variance Decomposition")
    plt.savefig("bias_variance")


In [ ]:
from scripts.proj1_helpers import *

def bias_variance(y, x):
    
    seeds = range(10)
    ratio_train = 0.01
    degrees = range(15)
    # define list to store the variable
    rmse_tr = np.empty((len(seeds), len(degrees)))
    rmse_te = np.empty((len(seeds), len(degrees)))
    
    for index_seed, seed in enumerate(seeds):
        np.random.seed(seed)

        # Split data
        y_tr, x_tr, y_te, x_te = split_data(y, x, ratio_train, seed)
        
        for degree in degrees:
            tx_tr = build_poly(x_tr, degree)
            tx_te = build_poly(x_te, degree)

            w = least_squares(y_tr, tx_tr)
            
            loss_tr = compute_mse(y_tr, tx_tr, w)
            loss_te = compute_mse(y_te, tx_te, w)
            
            rmse_tr[index_seed, degree-1] = np.sqrt(2*loss_tr)
            rmse_te[index_seed, degree-1] = np.sqrt(2*loss_te)

    return degrees, rmse_tr, rmse_te

In [ ]:
degrees, rmse_tr, rmse_te = bias_variance(y, tx)


In [ ]:
print(rmse_tr[-1], rmse_te[-1])

In [ ]:
bias_variance_decomposition_visualization(degrees, rmse_tr, rmse_te)